<a href="https://colab.research.google.com/github/ibmkeiba/1_WebScraper/blob/main/netkeiba_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **【netkeiba.com】 本番 Stage4 #2**

> **今月分のレース一覧とレース情報/結果を自動的に取得(SQLite出力)**

* urllib, selenium(chromium) でアクセス
* BeaurifulSoupで解析
* レースカレンダーからレースがある日を取得 

    【取得するレース情報】（16）
        ymd, date, menu, place, r, title, start_time, distance, race_type, number, prize, weather, course_status, grade, race_id, url

    【取得するレース結果】（19）
        rank, waku, umaban, bamei, bamei_url, sexage, handicap, jockey, jockey_url, racetime, difference, popular, odds, furlong3, corner_rank, stable, trainer_url, weight, race_id

> **参考URL**

* 【netkeiba.com】トップページ (https://www.netkeiba.com/)  
* 【netkeiba.com】開催一覧　(https://race.netkeiba.com/top/calendar.html)  
* 【netkeiba.com】開催一覧 (年月指定) (https://race.netkeiba.com/top/calendar.html?year=2021&month=8)
* 【netkeiba.com】開催レース一覧（特定日） (https://race.netkeiba.com/top/race_list.html?kaisai_date=20210801)

## **初期設定**

### **ライブラリインポート**

In [2]:
# choromiumとseleniumをインストール
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [68.5 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [3]:
# ライブラリのインポート
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [4]:
# ライブラリのインポート
from urllib import request as req
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time
import shutil
import sqlite3

## **ユーザー関数**

### **①レース開催日の取得**

In [60]:
# URLからソースコードを取得する関数 <urllib>  【url > soup】
def my_request(url):

    # URLを開く
    res = req.urlopen(url)

    # URLのソースコードを取得
    soup = BeautifulSoup(res, 'html.parser')

    # 暗黙的待機（サーバーの負荷軽減目的）
    time.sleep(0.1)
    
    return soup

In [61]:
# 開催一覧のソースコードから選択中の年月を抽出する関数 【soup > 表示している開催一覧ページの年月（文字列型）】

def my_selected_ym(soup):

    # 選択中の年 > 例 2021
    selected_y = soup.select_one('select#cal_select_year option[selected@selected]').text

    # 選択中の月 > 例 8
    selected_m = soup.select_one('div.RaceNumWrap.CalendarMonth ul li.Active a').text.replace('月', '')

    # 選択中の年月の6桁文字表記 例 202109
    selected_ym = selected_y + selected_m.zfill(2)

    return selected_ym

In [62]:
# 開催一覧のURLから現在の年月と開催がある日を取得 【url > ym（文字列型）, days（整数のリスト型）】
def my_get_raceday_from_url(url_calendar='https://race.netkeiba.com/top/calendar.html', ymd_list=None):

    # URLのソースコードを取得
    soup = my_request(url_calendar)

    # 表示中の年月を取得
    ym = my_selected_ym(soup)

    #レース開催がある日のエレメント（なければ []）
    race_day_elements = soup.select('table.Calendar_Table td.RaceCellBox a')

    # 開催レースが月に一度もなければこれまでの開催日リストを返す
    if race_day_elements == []:
        return ymd_list

    # ymd_listがなければ初期設定
    ymd_list = [] if ymd_list is None else ymd_list
    
    # レース開催がある日を回す
    for ele in race_day_elements:
        
        # レース開催日 例 5
        day = int(ele.select_one('span.Day').text)

        # リストに追加
        ymd_list.append(ym + str(day).zfill(2))

    # 翌月の６桁文字表記（文字列型）
    ym_next = str(int(ym[:4]) + int(ym[4:])//12) + str(int(ym[4:])%12+1).zfill(2)

    # 翌月の開催一覧カレンダーURL
    url_calendar_next = 'https://race.netkeiba.com/top/calendar.html?year=' + ym_next[:4] + '&month=' + str(int(ym_next[4:]))

    return my_get_raceday_from_url(url_calendar_next, ymd_list)

### **②開催レース一覧**

In [63]:
# ドライバーを起動 【 - > driver 】
def my_start_driver():

    # オプション設定（★　Colabでそのまま使用できる設定 ★）
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Colabではヘッドレス必須*
    options.add_argument('--no-sandbox') # Colab環境でのヘッドレスで必須*
    options.add_argument('--disable-gpu') # soup.prettify() の実行に必要
    options.add_argument('--disable-dev-shm-usage') # メモリ不足回避に有効

    # ドライバーを起動
    driver = webdriver.Chrome('chromedriver', options=options)

    # 要素が見つかるまでの待機時間(秒)を設定
    driver.implicitly_wait(5)

    return driver

In [64]:
# 指定URLのソースコードを取得 【 url > soup 】
def my_get_source_driver(url):

    global driver

    # もしグローバル変数にdriverがなければ起動
    try:
        driver.current_url
    except:
        driver = my_start_driver()
        print('■ driverを起動しました\n')

    # ブラウザでURLにアクセス
    driver.get(url)

    # ソースコードをBeautifulSopuに変換
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    return soup

In [65]:
# 目的のレースのエレメントから情報を取得
def my_race_info(race_element):

    # レースURL 'https://', race_url
    race_url = race_element.select_one('a.').attrs['href'].replace('..', 'https://race.netkeiba.com')

    # レースID
    race_id = race_url.split('?race_id=')[1].split('&')[0]

    # レース番 '1R', r
    r = race_element.select_one('a. div.Race_Num span').text

    # レースタイトル '3歳未勝利', title
    title = race_element.select_one('a. span.ItemTitle').text

    # 開始時刻 '10:10 ', time
    start_time = race_element.select_one('a. span.RaceList_Itemtime').text

    # コースタイプと距離 'ダ1200m', distance
    #distance = race_element.select_one('a. span.RaceList_ItemLong').text
    
    try:
        distance = race_element.select_one('a. span.RaceList_ItemLong').text

    except AttributeError:
        distance = race_element.select('a. div.RaceData span')[1].text
    
    except:
        print('◆ Select error, ', 'distance ↓')
        distance = ''

    # 出場馬数 '16頭', number
    number = race_element.select_one('a. span.RaceList_Itemnumber').text.replace(' ','')

    # レースグレード, '3', grade
    try:
        grade = race_element.select_one('a. span.Icon_GradeType').attrs['class']
    except: # 存在しない場合
        grade = ''
    else: # 存在する場合
        grade = grade[1].replace('Icon_GradeType', '')

    return [r, race_id, start_time, title, grade, number, distance, race_url]

In [66]:
# ②開催レース一覧のソースコードから全レースの情報を取得 【 soup > DataFrame 】
def my_race_list(soup):

    # 1日分のレース情報を格納するリスト
    races_list = []

    # 開催年月日（ymd） 文字列型
    ymd = soup.select_one('ul#date_list_sub li.Active').attrs['date']

    # 開催日（曜日）
    date = soup.select_one('ul#date_list_sub li.Active a').attrs['title']

    # 開催場所のエレメントリスト > 3箇所
    place_elelments = soup.select('div.RaceList_Box > dl.RaceList_DataList')

    # 開催場所を回す
    for i, place_ele in enumerate(place_elelments):

        # 開催場所
        place = place_ele.select_one('p.RaceList_DataTitle').text.split(' ')[1]

        # レース一覧のエレメントリスト
        race_elements = place_ele.select('dd.RaceList_Data ul li')

        # レース会場名とレース数を表示
        print('・{}年{}　> {} 【{}】 {} レース'.format(ymd[:4], date, i+1, place, len(race_elements)))

        # レースを回す
        for j, race_ele in enumerate(race_elements):

            # レースのエレメントから情報を取得
            race_list = my_race_info(race_ele)
            # レース情報を表示
            print(race_list)

            # 1日分のレース情報に追加
            races_list.append([int(ymd), date, place] + race_list)
            
    # 1日分のレース情報をDataFrame形式に変換
    ymd_df = pd.DataFrame(races_list, index=None, columns=['ymd', 'date', 'place', 'r', 'race_id', 'start_time', 'title', 'grade', 'number', 'distance', 'url'])

    return ymd_df

### **③レース出馬表・結果**

In [67]:
# gradeが指定したgradeか判定
def my_grade_check(grade, grade_lim=None):

    # グレード指定がない場合
    if grade_lim == None:
        return 'pass'
    # グレード指定がある場合
    elif grade == '':
        return 'skip'
    elif int(grade) > grade_lim:
        return 'skip'
    else:
        return 'pass'

In [68]:
# 出馬表/結果・払戻からレースの情報を取得 【 soup > list 】
def my_result1(soup):

    # 選択されているサブメニュー（出馬表 / 結果・払戻）
    menu = soup.select_one('ul.RaceMainMenu li a.Active').attrs['title']

    # レース開催日
    date = soup.select_one('dl#RaceList_DateList > dd.Active a').text

    # 開催場所
    try:
        place = soup.select_one('div.RaceKaisaiWrap ul li.Active a').text
    except:
        place = soup.select('div.RaceList_NameBox div.RaceData02 span')[1].text

    # レース番号
    race_num = soup.select_one('div.RaceList_NameBox span.RaceNum').text

    # レース名
    title = soup.select_one('div.RaceList_NameBox div.RaceName').text.replace('\n', '')

    # 発走時刻
    start_time = soup.select_one('div.RaceList_NameBox div.RaceData01').text.replace('\n', '').split('発走')[0]

    # 距離
    distance = soup.select_one('div.RaceList_NameBox div.RaceData01 span').text

    # レース種別 (要注意)
    race_type = soup.select('div.RaceList_NameBox div.RaceData02 span')[3].text

    # 出馬数
    number = soup.select('div.RaceList_NameBox div.RaceData02 span')[7].text

    # 賞金金額[万円]
    prize = soup.select('div.RaceList_NameBox div.RaceData02 span')[8].text.replace('本賞金:', '').replace('万円', '').replace(',', '')

    result1_list = [date, menu, place, race_num, title, start_time, distance, race_type, number, prize]

    # 選択されているメニューが'結果・払戻'の場合
    if menu == '結果・払戻':

        # 天候 (結果・払戻 限定)
        weather = soup.select_one('div.RaceList_NameBox div.RaceData01').text.split('天候:')[1].split('\n')[0]

        # 馬場の状態 (結果・払戻 限定)
        course_status = soup.select('div.RaceList_NameBox div.RaceData01 span')[-1].text.split('馬場:')[1]

        result1_list = result1_list + [weather, course_status]

    # 選択されているメニューが'出馬表'の場合
    else:
        result1_list = result1_list + ['', '']

    return result1_list

In [69]:
# 【結果・払戻】の１頭分のエレメントから、情報を取得するユーザ関数
def my_result2a(horse_ele):

    # 【結果・払戻】の辞書
    result_dic = {}

    # 着順 rank
    try:
        result_dic['rank'] = int(horse_ele.select_one('td.Result_Num > div.Rank').text)
    except:
        return '取り消し'

    # 枠 waku
    result_dic['waku'] = horse_ele.select('td.Num div')[0].text

    # 馬番 umaban
    result_dic['umaban'] = horse_ele.select('td.Num div')[1].text

    # 馬名 bamei
    result_dic['bamei'] = horse_ele.select_one('td.Horse_Info span.Horse_Name a').text

    # 馬のURL bamei_url
    result_dic['bamei_url'] = horse_ele.select_one('td.Horse_Info span.Horse_Name a').attrs['href']

    # 性齢 sexage
    result_dic['sexage'] = horse_ele.select_one('td.Horse_Info span.Lgt_Txt').text.replace('\n', '')

    # 斤量 handicap
    result_dic['handicap'] = horse_ele.select_one('span.JockeyWeight').text

    try:
        # 騎手 jockey
        result_dic['jockey'] = horse_ele.select_one('td.Jockey a').text.replace(' ', '')

        # 騎手のURL jockey_url
        result_dic['jockey_url'] = horse_ele.select_one('td.Jockey a').attrs['href']
    except:
        # 騎手 jockey
        result_dic['jockey'] = horse_ele.select_one('td.Jockey').text.replace('\n', '').replace(' ', '')

        # 騎手のURL jockey_url
        result_dic['jockey_url'] = ''   

    # タイム racetime
    result_dic['racetime'] = horse_ele.select_one('span.RaceTime').text

    # 着差 difference
    result_dic['difference'] = horse_ele.select('span.RaceTime')[1].text

    # 人気 popular
    result_dic['popular'] = horse_ele.select_one('span.OddsPeople').text

    # 単勝オッズ odds
    result_dic['odds'] = horse_ele.select('td.Odds span')[1].text

    # 後3F　furlong3
    result_dic['furlong3'] = horse_ele.select('td.Time')[2].text.replace('\n', '')

    # コーナー通過順 corner_rank
    result_dic['corner_rank'] = horse_ele.select_one('td.PassageRate').text.replace('\n', '')

    # 厩舎(施設名) stable
    result_dic['stable'] = horse_ele.select_one('td.Trainer span').text

    # 厩舎(調教師) trainer
    result_dic['trainer'] = horse_ele.select_one('td.Trainer a').text

    # 厩舎(調教師)のURL trainer_url
    result_dic['trainer_url'] = horse_ele.select_one('td.Trainer a').attrs['href']

    # 馬体重 weight
    result_dic['weight'] = horse_ele.select_one('td.Weight').text.replace('\n', '').split('(')[0]

    # 馬体重の増減 weight_change
    try:
        result_dic['weight_change'] = horse_ele.select_one('td.Weight').text.replace('\n', '').split('(')[1].split(')')[0]
    except:
        print('◆ 注意： "馬体重の増減" の取得に失敗しました')

    # 取得した情報の表示
    #print(list(result_dic.values()))

    return list(result_dic.values())

In [70]:
# 【出馬表】の１頭分のエレメントから、情報を取得するユーザ関数
def my_result2b(horse_ele):

    # 【出馬表】の辞書
    result_dic = {}

    # 着順 rank
    result_dic['rank'] = ''

    # 枠 waku
    result_dic['waku'] = horse_ele.select('td span')[0].text

    # 馬番 umaban
    result_dic['umaban'] = horse_ele.select('td')[1].text

    # 馬名 bamei
    result_dic['bamei'] = horse_ele.select_one('td span.HorseName a').attrs['title']

    # 馬のURL bamei_url
    result_dic['bamei_url'] = horse_ele.select_one('td span.HorseName a').attrs['href']

    # 性齢 sexage
    result_dic['sexage'] = horse_ele.select_one('td.Barei').text

    # 斤量 handicap
    result_dic['handicap'] = horse_ele.select('td')[5].text

    try:
        # 騎手 jockey
        result_dic['jockey'] = horse_ele.select_one('td.Jockey a').attrs['title']

        # 騎手のURL jockey_url
        result_dic['jockey_url'] = horse_ele.select_one('td.Jockey a').attrs['href']
    except:
        # 騎手 jockey
        result_dic['jockey'] = element.select_one('td.Jockey').text.replace('\n','').replace(' ','')

        # 騎手のURL jockey_url
        result_dic['jockey_url'] = ''

    # タイム racetime
    result_dic['racetime'] = ''

    # 着差 difference
    result_dic['difference'] = ''
    
    # 人気 popular
    result_dic['popular'] = horse_ele.select('td.Popular span')[1].text

    # 単勝オッズ odds
    result_dic['odds'] = horse_ele.select('td.Popular span')[0].text

    # 後3F　furlong3
    result_dic['furlong3'] = ''

    # コーナー通過順 corner_rank
    result_dic['corner_rank'] = ''

    # 厩舎(施設名) stable
    result_dic['stable'] = horse_ele.select_one('td.Trainer span').text

    # 厩舎(調教師) trainer
    result_dic['trainer'] = horse_ele.select_one('td.Trainer a').attrs['title']

    # 厩舎(調教師)のURL trainer_url
    result_dic['trainer_url'] = horse_ele.select_one('td.Trainer a').attrs['href']

    # 馬体重 weight
    result_dic['weight'] = horse_ele.select_one('td.Weight').text.replace('\n', '').split('(')[0]

    # 馬体重の増減 weight_change
    try:
        result_dic['weight_change'] = horse_ele.select_one('td.Weight').text.replace('\n', '').split('(')[1].split(')')[0]
    except:
        result_dic['weight_change'] = ''

    # 取得した情報の表示
    #print(list(result_dic.values()))

    return list(result_dic.values())

In [71]:
# 出馬表 or 結果・払戻を判定してレースの結果を取得 【 horse_ele > def > list 】
def my_result2(horse_ele, menu):

    # 選択されているサブメニューが "結果・払戻" の場合
    if menu == '結果・払戻':
        return my_result2a(horse_ele)

    # 選択されているサブメニューが "出馬表" の場合
    elif menu == '出馬表':
        return my_result2b(horse_ele)

## **メインプログラム**

### **①レース開催日の取得**

In [72]:
# 指定年月URLからレース開催がある年月日を取得
ymd_list = my_get_raceday_from_url()
print(ymd_list)

['20210904', '20210905', '20210911', '20210912', '20210918', '20210919', '20210920', '20210925', '20210926', '20211002', '20211003']


In [73]:
ymd_df = pd.DataFrame(ymd_list, columns=['ymd'])
ymd_df

,ymd
0,20210904
1,20210905
2,20210911
3,20210912
4,20210918
5,20210919
6,20210920
7,20210925
8,20210926
9,20211002


In [74]:
ymd_df['ymd'].tolist()

['20210904',
 '20210905',
 '20210911',
 '20210912',
 '20210918',
 '20210919',
 '20210920',
 '20210925',
 '20210926',
 '20211002',
 '20211003']

### **②開催レース一覧**

In [75]:
# 全レース情報を格納する空のDataFrameを作成
races_df = pd.DataFrame()

# 日を回す d[1:index, 2:d]
for ymd in ymd_list:

    # レース開催日のURLを作成
    url_ymd = 'https://race.netkeiba.com/top/race_list.html?kaisai_date=' + ymd
    print('\n\n①', ymd, url_ymd, '\n')

    # 全ての会場とレースの情報を取得

    # URL（開催日）のソースコードを取得（selenium）
    soup = my_get_source_driver(url_ymd)

    # 特定日の全レース情報を取得
    race_df = my_race_list(soup)

    # データフレームに追加
    races_df = races_df.append(race_df).reset_index(drop=True)



① 20210904 https://race.netkeiba.com/top/race_list.html?kaisai_date=20210904 

・2021年9月4日(土)　> 1 【新潟】 12 レース
['1R', '202104040701', '10:10 ', '3歳以上障害未勝利', '', '13頭', '障2850m', 'https://race.netkeiba.com/race/result.html?race_id=202104040701&rf=race_list']
['2R', '202104040702', '10:45 ', '2歳未勝利', '', '15頭', 'ダ1800m', 'https://race.netkeiba.com/race/result.html?race_id=202104040702&rf=race_list']
['3R', '202104040703', '11:15 ', '3歳未勝利', '', '15頭', 'ダ1800m', 'https://race.netkeiba.com/race/result.html?race_id=202104040703&rf=race_list']
['4R', '202104040704', '11:45 ', '3歳以上障害OP', '5', '11頭', '障3250m', 'https://race.netkeiba.com/race/result.html?race_id=202104040704&rf=race_list']
['5R', '202104040705', '12:35 ', '2歳新馬', '', '10頭', '芝2000m', 'https://race.netkeiba.com/race/result.html?race_id=202104040705&rf=race_list']
['6R', '202104040706', '13:05 ', '3歳未勝利', '', '18頭', '芝1600m', 'https://race.netkeiba.com/race/result.html?race_id=202104040706&rf=race_list']
['7R', '202104040707', '

In [76]:
races_df

,ymd,date,place,r,race_id,start_time,title,grade,number,distance,url
0,20210904,9月4日(土),新潟,1R,202104040701,10:10,3歳以上障害未勝利,,13頭,障2850m,https://race.netkeiba.com/race/result.html?rac...
1,20210904,9月4日(土),新潟,2R,202104040702,10:45,2歳未勝利,,15頭,ダ1800m,https://race.netkeiba.com/race/result.html?rac...
2,20210904,9月4日(土),新潟,3R,202104040703,11:15,3歳未勝利,,15頭,ダ1800m,https://race.netkeiba.com/race/result.html?rac...
3,20210904,9月4日(土),新潟,4R,202104040704,11:45,3歳以上障害OP,5,11頭,障3250m,https://race.netkeiba.com/race/result.html?rac...
4,20210904,9月4日(土),新潟,5R,202104040705,12:35,2歳新馬,,10頭,芝2000m,https://race.netkeiba.com/race/result.html?rac...
...,...,...,...,...,...,...,...,...,...,...,...
248,20211003,10月3日(日),中山,11R,202106040911,15:40,スプリンター,1,22頭,芝1200m,https://race.netkeiba.com/race/shutuba.html?ra...
249,20211003,10月3日(日),中山,12R,202106040912,,外房S,16,35頭,ダ1200m,https://race.netkeiba.com/race/shutuba.html?ra...
250,20211003,10月3日(日),中京,9R,202107050909,,浜名湖特別,17,15頭,芝2000m,https://race.netkeiba.com/race/shutuba.html?ra...
251,20211003,10月3日(日),中京,10R,202107050910,14:50,白川郷S,16,27頭,ダ1900m,https://race.netkeiba.com/race/shutuba.html?ra...


### **③レース出馬表・結果**

In [77]:
# ③　レース出馬表・結果

# 全レースの情報のリスト
results1_list = []

# 全レースの結果・払戻のリスト
results2_list = []

# レースを回す df[index, row[数値], row['列名']]
for index, row in races_df.iterrows():

    print('③ {:<5}\t{:<10}\t{:<3}\t{}\t{:<7}\t{:<10}\t\t> '.format(index, row['date'], row['place'], row['grade'], row['r'], row['title']), end='')

    # レースのソースコード取得
    soup = my_request(row['url'])

    # 選択されているサブメニュー（出馬表 / 結果・払戻）
    menu = soup.select_one('ul.RaceMainMenu li a.Active').attrs['title']

    # URLが "結果・払戻" の場合
    if menu == '結果・払戻':

        # レース結果・払戻のトップエレメント
        horse_elements = soup.select('div.ResultTableWrap tr.HorseList')

    # URLが "出馬表" の場合
    elif menu == '出馬表':

        # レースのソースコード取得
        soup = my_get_source_driver(row['url'])

        # レース出馬表のトップエレメント
        horse_elements = soup.select('div.RaceTableArea tr.HorseList')

    # 1. レースの情報を取得　（1レース） ★
    result1_list = my_result1(soup)
    print(result1_list, row['url'])

    # 1. 全レースの情報のリストに追加
    results1_list.append([row['ymd']] + result1_list + [row['grade'], row['race_id'], row['url']])

    # 2. レースの結果・払戻を取得 （1レース）

    # 全馬のレース結果・払戻を取得
    for i, element in enumerate(horse_elements):

        # レース結果・払戻を取得 （1頭） ★
        result2_list = my_result2(element, menu)

        # 取得したレース結果（1頭分）の表示
        #print('\t#{}\t> {}'.format(i+1, result2_list))

        # 2. 全レースの結果・払戻のリストに追加　（完走馬のみ）
        if type(result2_list) == list:
            results2_list.append(result2_list + [row['race_id']])

    # 2. レースの結果・払戻を取得完了 （1レース） ★
    print('')

# ③-1. 全レースの情報のDataFrame
columns1 = ['ymd', 'date', 'menu', 'place', 'r', 'title', 'start_time', 'distance', 'race_type', 'number', 'prize', 'weather', 'course_status', 'grade', 'race_id', 'url']
results1_df = pd.DataFrame(results1_list, index=None, columns=columns1)

# ③-2. 全レースの結果・払戻のDataFrame
columns2 = ['rank', 'waku', 'umaban', 'bamei', 'bamei_url', 'sexage', 'handicap', 'jockey', 'jockey_url', 'racetime', 'difference', 'popular', 'odds', 'furlong3', 'corner_rank', 'stable', 'trainer', 'trainer_url', 'weight', 'weight_change', 'race_id']
results2_df = pd.DataFrame(results2_list, index=None, columns=columns2)

③ 0    	9月4日(土)   	新潟 		1R     	3歳以上障害未勝利 		> ['9月4日(土)', '結果・払戻', '新潟', '1R', '3歳以上障害未勝利', '10:10', ' 障2850m', '障害３歳以上', '13頭', '79032020012079', '曇', '稍'] https://race.netkeiba.com/race/result.html?race_id=202104040701&rf=race_list

③ 1    	9月4日(土)   	新潟 		2R     	2歳未勝利     		> ['9月4日(土)', '結果・払戻', '新潟', '2R', '2歳未勝利', '10:45', ' ダ1800m', 'サラ系２歳', '15頭', '5102001307751', '曇', '稍'] https://race.netkeiba.com/race/result.html?race_id=202104040702&rf=race_list

③ 2    	9月4日(土)   	新潟 		3R     	3歳未勝利     		> ['9月4日(土)', '結果・払戻', '新潟', '3R', '3歳未勝利', '11:15', ' ダ1800m', 'サラ系３歳', '15頭', '5102001307751', '小雨', '稍'] https://race.netkeiba.com/race/result.html?race_id=202104040703&rf=race_list

③ 3    	9月4日(土)   	新潟 	5	4R     	3歳以上障害OP  		> ['9月4日(土)', '結果・払戻', '新潟', '4R', '3歳以上障害OP', '11:45', ' 障3250m', '障害３歳以上', '11頭', '1350540340200135', '曇', '稍'] https://race.netkeiba.com/race/result.html?race_id=202104040704&rf=race_list

③ 4    	9月4日(土)   	新潟 		5R     	2歳新馬      		> ['9月4日(土)', '結果・払戻', '新

In [78]:
results1_df

,ymd,date,menu,place,r,title,start_time,distance,race_type,number,prize,weather,course_status,grade,race_id,url
0,20210904,9月4日(土),結果・払戻,新潟,1R,3歳以上障害未勝利,10:10,障2850m,障害３歳以上,13頭,79032020012079,曇,稍,,202104040701,https://race.netkeiba.com/race/result.html?rac...
1,20210904,9月4日(土),結果・払戻,新潟,2R,2歳未勝利,10:45,ダ1800m,サラ系２歳,15頭,5102001307751,曇,稍,,202104040702,https://race.netkeiba.com/race/result.html?rac...
2,20210904,9月4日(土),結果・払戻,新潟,3R,3歳未勝利,11:15,ダ1800m,サラ系３歳,15頭,5102001307751,小雨,稍,,202104040703,https://race.netkeiba.com/race/result.html?rac...
3,20210904,9月4日(土),結果・払戻,新潟,4R,3歳以上障害OP,11:45,障3250m,障害３歳以上,11頭,1350540340200135,曇,稍,5,202104040704,https://race.netkeiba.com/race/result.html?rac...
4,20210904,9月4日(土),結果・払戻,新潟,5R,2歳新馬,12:35,芝2000m,サラ系２歳,10頭,70028018011070,曇,稍,,202104040705,https://race.netkeiba.com/race/result.html?rac...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,20211003,10月3日(日),出馬表,中山,11R,スプリンターズS,15:40,芝1200m,サラ系３歳以上,22頭,130005200330020001300,,,1,202106040911,https://race.netkeiba.com/race/shutuba.html?ra...
249,20211003,10月3日(日),出馬表,中山,12R,外房ステークス,ダ1200m (右),ダ1200m,サラ系３歳以上,35頭,1820730460270182,,,16,202106040912,https://race.netkeiba.com/race/shutuba.html?ra...
250,20211003,10月3日(日),出馬表,中京,9R,浜名湖特別,芝2000m (左),芝2000m,サラ系３歳以上,15頭,1500600380230150,,,17,202107050909,https://race.netkeiba.com/race/shutuba.html?ra...
251,20211003,10月3日(日),出馬表,中京,10R,白川郷ステークス,14:50,ダ1900m,サラ系３歳以上,27頭,1820730460270182,,,16,202107050910,https://race.netkeiba.com/race/shutuba.html?ra...


In [79]:
results2_df

,rank,waku,umaban,bamei,bamei_url,sexage,handicap,jockey,jockey_url,racetime,difference,popular,odds,furlong3,corner_rank,stable,trainer,trainer_url,weight,weight_change,race_id
0,1,1,1,メイショウギガース,https://db.netkeiba.com/horse/2014102973,牡7,60.0,中村,https://db.netkeiba.com/jockey/01101/,3:06.3,,5,18.9,13.1,1-1-1-1,栗東,岡田,https://db.netkeiba.com/trainer/01066/,490,+6,202104040701
1,2,2,2,リッジマン,https://db.netkeiba.com/horse/2013103569,牡8,60.0,森一,https://db.netkeiba.com/jockey/01139/,3:06.4,1/2,1,1.2,13.1,6-6-7-7,栗東,庄野,https://db.netkeiba.com/trainer/01087/,462,+2,202104040701
2,3,5,7,アナンダライト,https://db.netkeiba.com/horse/2017105631,セ4,60.0,五十嵐,https://db.netkeiba.com/jockey/01069/,3:07.1,4,3,8.8,13.1,3-3-3-3,美浦,田村,https://db.netkeiba.com/trainer/01027/,510,-2,202104040701
3,4,4,5,クリエイトザライフ,https://db.netkeiba.com/horse/2016105987,セ5,60.0,北沢,https://db.netkeiba.com/jockey/00691/,3:07.5,2,4,17.2,13.2,4-4-3-3,栗東,吉岡,https://db.netkeiba.com/trainer/01176/,482,+2,202104040701
4,5,3,3,レーヴドオルフェ,https://db.netkeiba.com/horse/2017102082,牡4,60.0,小野寺,https://db.netkeiba.com/jockey/01123/,3:07.6,3/4,2,8.3,13.2,8-8-5-5,美浦,菊川,https://db.netkeiba.com/trainer/01052/,500,-6,202104040701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3464,,,,レッドガラン,https://db.netkeiba.com/horse/2015104417,牡6,57.0,斎藤,https://db.netkeiba.com/jockey/01178/,,,12,63.4,,,栗東,安田隆,https://db.netkeiba.com/trainer/00438/,,,202107050911
3465,,,,レッドヴェイロン,https://db.netkeiba.com/horse/2015104714,牡6,56.0,亀田,https://db.netkeiba.com/jockey/01176/,,,10,46.1,,,美浦,鹿戸,https://db.netkeiba.com/trainer/01097/,,,202107050911
3466,,,,ロフティフレーズ,https://db.netkeiba.com/horse/2015103535,牝6,54.0,○○,,,,26,240.3,,,美浦,上原,https://db.netkeiba.com/trainer/00423/,,,202107050911
3467,,,,ワールドウインズ,https://db.netkeiba.com/horse/2017104093,セ4,56.0,武豊,https://db.netkeiba.com/jockey/00666/,,,14,82.7,,,栗東,武幸,https://db.netkeiba.com/trainer/01160/,,,202107050911


In [80]:
results1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ymd            253 non-null    int64 
 1   date           253 non-null    object
 2   menu           253 non-null    object
 3   place          253 non-null    object
 4   r              253 non-null    object
 5   title          253 non-null    object
 6   start_time     253 non-null    object
 7   distance       253 non-null    object
 8   race_type      253 non-null    object
 9   number         253 non-null    object
 10  prize          253 non-null    object
 11  weather        253 non-null    object
 12  course_status  253 non-null    object
 13  grade          253 non-null    object
 14  race_id        253 non-null    object
 15  url            253 non-null    object
dtypes: int64(1), object(15)
memory usage: 31.8+ KB


### **SQLite出力**

In [81]:
# データベース（db1）を作成（既に存在していれば接続）
conn = sqlite3.connect('dbkeiba')

In [82]:
# SQLiteを操作するためのカーソルを作成
cur = conn.cursor()

In [150]:
# テーブル（table1）が存在していれば削除
cur.execute('DROP TABLE IF EXISTS info')

# テーブル（info）を作成
cur.execute("""CREATE TABLE info(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ymd INT, 
    date TEXT, 
    menu TEXT, 
    place TEXT, 
    r TEXT, 
    title TEXT, 
    start_time TEXT, 
    distance TEXT, 
    race_type TEXT, 
    number TEXT, 
    prize TEXT, 
    weather TEXT, 
    course_status TEXT, 
    grade TEXT, 
    race_id TEXT, 
    url TEXT)
""")

#### 1レコードの追加

In [152]:
# 1レース分のレース情報のリストを作成
data = results1_df.copy().iloc[0].tolist()

# ymdをnumpy.int64からint型に変換
data[0] = int(data[0])

# データ登録4
cur.execute('INSERT INTO info (ymd, date, menu, place, r, title, start_time, distance, race_type, number, prize, weather, course_status, grade, race_id, url) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', data)

# コミット
conn.commit()

In [153]:
# データの読み取り（全レコード）
cur.execute('SELECT * FROM info')

# 結果の表示
print(cur.fetchall())

[(1, 20210904, '9月4日(土)', '結果・払戻', '新潟', '1R', '3歳以上障害未勝利', '10:10', ' 障2850m', '障害３歳以上', '13頭', '79032020012079', '曇', '稍', '', '202104040701', 'https://race.netkeiba.com/race/result.html?race_id=202104040701&rf=race_list')]


In [103]:
df_db = results1_df.copy()
df_db

,ymd,date,menu,place,r,title,start_time,distance,race_type,number,prize,weather,course_status,grade,race_id,url
1,20210904,9月4日(土),結果・払戻,新潟,2R,2歳未勝利,10:45,ダ1800m,サラ系２歳,15頭,5102001307751,曇,稍,,202104040702,https://race.netkeiba.com/race/result.html?rac...
2,20210904,9月4日(土),結果・払戻,新潟,3R,3歳未勝利,11:15,ダ1800m,サラ系３歳,15頭,5102001307751,小雨,稍,,202104040703,https://race.netkeiba.com/race/result.html?rac...
3,20210904,9月4日(土),結果・払戻,新潟,4R,3歳以上障害OP,11:45,障3250m,障害３歳以上,11頭,1350540340200135,曇,稍,5,202104040704,https://race.netkeiba.com/race/result.html?rac...


In [154]:
# テーブル（table1）が存在していれば削除
cur.execute('DROP TABLE IF EXISTS db2')

# DataFrameをまとめてデータベースに追加
results1_df.to_sql('db2', conn, if_exists='replace', index=None, method='multi')

In [155]:
# データの読み取り（全レコード）
result = cur.execute('SELECT * FROM db2').fetchall()

# 結果の表示
print(result)

[(20210904, '9月4日(土)', '結果・払戻', '新潟', '1R', '3歳以上障害未勝利', '10:10', ' 障2850m', '障害３歳以上', '13頭', '79032020012079', '曇', '稍', '', '202104040701', 'https://race.netkeiba.com/race/result.html?race_id=202104040701&rf=race_list'), (20210904, '9月4日(土)', '結果・払戻', '新潟', '2R', '2歳未勝利', '10:45', ' ダ1800m', 'サラ系２歳', '15頭', '5102001307751', '曇', '稍', '', '202104040702', 'https://race.netkeiba.com/race/result.html?race_id=202104040702&rf=race_list'), (20210904, '9月4日(土)', '結果・払戻', '新潟', '3R', '3歳未勝利', '11:15', ' ダ1800m', 'サラ系３歳', '15頭', '5102001307751', '小雨', '稍', '', '202104040703', 'https://race.netkeiba.com/race/result.html?race_id=202104040703&rf=race_list'), (20210904, '9月4日(土)', '結果・払戻', '新潟', '4R', '3歳以上障害OP', '11:45', ' 障3250m', '障害３歳以上', '11頭', '1350540340200135', '曇', '稍', '5', '202104040704', 'https://race.netkeiba.com/race/result.html?race_id=202104040704&rf=race_list'), (20210904, '9月4日(土)', '結果・払戻', '新潟', '5R', '2歳新馬', '12:35', ' 芝2000m', 'サラ系２歳', '10頭', '70028018011070', '曇', '稍', '', '

In [156]:
# カラム（フィールド：列）一覧を表示する【 "PRAGMA TABLE_INFO(テーブル名) "】
columns_info = cur.execute("PRAGMA TABLE_INFO(db2)").fetchall()

# カラム名のリスト
columns = np.array(columns_info)[:,1].tolist()

# データフレームとして表示
pd.DataFrame(result, columns=columns)

,ymd,date,menu,place,r,title,start_time,distance,race_type,number,prize,weather,course_status,grade,race_id,url
0,20210904,9月4日(土),結果・払戻,新潟,1R,3歳以上障害未勝利,10:10,障2850m,障害３歳以上,13頭,79032020012079,曇,稍,,202104040701,https://race.netkeiba.com/race/result.html?rac...
1,20210904,9月4日(土),結果・払戻,新潟,2R,2歳未勝利,10:45,ダ1800m,サラ系２歳,15頭,5102001307751,曇,稍,,202104040702,https://race.netkeiba.com/race/result.html?rac...
2,20210904,9月4日(土),結果・払戻,新潟,3R,3歳未勝利,11:15,ダ1800m,サラ系３歳,15頭,5102001307751,小雨,稍,,202104040703,https://race.netkeiba.com/race/result.html?rac...
3,20210904,9月4日(土),結果・払戻,新潟,4R,3歳以上障害OP,11:45,障3250m,障害３歳以上,11頭,1350540340200135,曇,稍,5,202104040704,https://race.netkeiba.com/race/result.html?rac...
4,20210904,9月4日(土),結果・払戻,新潟,5R,2歳新馬,12:35,芝2000m,サラ系２歳,10頭,70028018011070,曇,稍,,202104040705,https://race.netkeiba.com/race/result.html?rac...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,20211003,10月3日(日),出馬表,中山,11R,スプリンターズS,15:40,芝1200m,サラ系３歳以上,22頭,130005200330020001300,,,1,202106040911,https://race.netkeiba.com/race/shutuba.html?ra...
249,20211003,10月3日(日),出馬表,中山,12R,外房ステークス,ダ1200m (右),ダ1200m,サラ系３歳以上,35頭,1820730460270182,,,16,202106040912,https://race.netkeiba.com/race/shutuba.html?ra...
250,20211003,10月3日(日),出馬表,中京,9R,浜名湖特別,芝2000m (左),芝2000m,サラ系３歳以上,15頭,1500600380230150,,,17,202107050909,https://race.netkeiba.com/race/shutuba.html?ra...
251,20211003,10月3日(日),出馬表,中京,10R,白川郷ステークス,14:50,ダ1900m,サラ系３歳以上,27頭,1820730460270182,,,16,202107050910,https://race.netkeiba.com/race/shutuba.html?ra...


### **ファイルをドライブにコピー**

In [ ]:
# ドライブコピー（ ② レース情報 ）

# コピー元のファイルパス
fp_info = 'RaceInfoTable_latest.csv'

# コピーファイル名
fp_info_copy = '/content/drive/My Drive/開発/1_WebScraper/netkeiba/data/' + fp_info

# SQLiteファイルをColabデフォルトディレクトリ内にコピー
shutil.copy(fp_info, fp_info_copy)

'/content/drive/My Drive/開発/1_WebScraper/netkeiba/data/RaceInfoTable_latest.csv'

In [ ]:
# ドライブコピー（ ③ レース結果 ）

# コピー元のファイルパス
fp_result = 'RaceResultTable_latest.csv'

# コピーファイル名
fp_result_copy = '/content/drive/My Drive/開発/1_WebScraper/netkeiba/data/' + fp_result

# SQLiteファイルをColabデフォルトディレクトリ内にコピー
shutil.copy(fp_result, fp_result_copy)

'/content/drive/My Drive/開発/1_WebScraper/netkeiba/data/RaceResultTable_latest.csv'